In [153]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager # Chrome driver 자동 업데이트
from selenium.webdriver.common.by import By # find element 사용
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait # 대기 시간 설정
from selenium.webdriver.support import expected_conditions as EC # 예외 상황 설정
# from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from tqdm import tqdm
import os
from urllib.request import urlretrieve
import time

chrome_options = Options()
chrome_options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지
chrome_options.add_experimental_option("useAutomationExtension", False) # 관리자모드 실행 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"]) # 불필요한 에러메시지 노출 방지

# Chrome driver Manager를 통해 크롬 드라이버 자동 설치
service = Service(excutable_path=ChromeDriverManager().install()) 

### test_01 | 뉴스 제목, 링크, 본문

In [154]:
# 사이트 오픈
naver_url = "http://www.naver.com/"
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.implicitly_wait(5) # 대기
driver.maximize_window() # 화면 최대화
driver.get(naver_url)

# 키워드 검색
search_keyword = "코난테크놀로지"
search_box = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div/div[3]/div[2]/div/form/fieldset/div/input")
search_box.send_keys(search_keyword)
search_box.send_keys(Keys.RETURN)

# 뉴스 탭 이동
driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/div[1]/section[4]/div/div[3]").click()
driver.implicitly_wait(5)

# 스크롤 n회 내리기
def scroll_down(n):
    scroll_actions = driver.find_element(By.CSS_SELECTOR, 'body')
    for i in range(n):
        scroll_actions.send_keys(Keys.END)
        driver.implicitly_wait(5)
scroll_down(30)

# 데이터프레임에 n개 뉴스 정보 저장
def crawling_news_list(n):
    news_df = pd.DataFrame(columns=["number", "title", "link", "news_contest"])
    for i in tqdm(range(n)):
        news_xpath_format = "/html/body/div[3]/div[2]/div/div[1]/section/div[1]/div[2]/ul/li[{}]/div[1]/div/div[2]/a[2]".format(i+1)
        try:
            news_title = driver.find_element(By.XPATH, news_xpath_format).text
            news_link = driver.find_element(By.XPATH, news_xpath_format).get_attribute("href")
            news_content = driver.find_element(By.XPATH, "/".join(news_xpath_format.split('/')[:-1]) + "/div/div/a").text    
        except: 
            news_xpath_format = "/html/body/div[3]/div[2]/div/div[1]/section/div[1]/div[2]/ul/li[{}]/div[1]/div/div[2]/a".format(i+1)
            news_title = driver.find_element(By.XPATH, news_xpath_format).text
            news_link = driver.find_element(By.XPATH, news_xpath_format).get_attribute("href")
            news_content = driver.find_element(By.XPATH, "/".join(news_xpath_format.split('/')[:-1]) + "/div/div/a").text  

        # news_df 행에 차례대로 저장
        news_df.loc[i] = [i, news_title, news_link, news_content]
    return news_df
news_df = crawling_news_list(20)
print(news_df.shape, news_df.head(), news_df.tail(), sep = "\n")

# 크롬 창 종료
driver.quit()

100%|██████████| 20/20 [00:10<00:00,  1.82it/s]


(20, 4)
   number                                           title  \
0       0                        에스넷시스템-코난테크놀로지, AI 사업 협력   
1       1                   인공지능 3D콘텐츠 제작 '코난보이스', 서비스 개편   
2       2  과기정통부, 카카오ㆍLG 등 AI 기업과 만남…“민관 협력 글로벌 경쟁력 확보...   
3       3   1위 폴라리스오피스 주가·2위 한글과컴퓨터 주가·3위 이스트소프트 주가, 3...   
4       4  [#IT라운지] 알바몬 '대학가 알바 현황'·카카오같이가치 '매달기부'·딥브레...   

                                                link  \
0  http://www.newsis.com/view/?id=NISX20240315_00...   
1              https://www.etnews.com/20240312000169   
2         https://www.etoday.co.kr/news/view/2337123   
3            https://cwn.kr/article/1065571503822755   
4  http://www.newsworks.co.kr/news/articleView.ht...   

                                        news_contest  
0  에스넷시스템이 코난테크놀로지와 인공지능(AI) 관련 사업 협력을 위한 업무협약(MO...  
1  코난테크놀로지(대표 김영섬)의 인공지능 성우 '코난보이스'가 3D캐릭터를 활용한 콘...  
2  이투데이=임유진 기자 | 과학기술정보통신부 강도현 제2차관은 5일 취임 후 첫 번째...  
3  또 6위 디지캡 주가는 트렌드지수 3,250점, 7위 MDS테크 주가는 트렌드지수 ...  
4  에스넷시스템이 코난

### test_02 | 뉴스 이미지

In [26]:
# 사이트 오픈
naver_url = "http://www.naver.com/"
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.implicitly_wait(5) # 대기
driver.maximize_window() # 화면 최대화
driver.get(naver_url)

# 키워드 검색
search_keyword = "코난테크놀로지"
search_box = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div/div[3]/div[2]/div/form/fieldset/div/input")
search_box.send_keys(search_keyword)
search_box.send_keys(Keys.RETURN)

# 뉴스 탭 이동
driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/div[1]/section[4]/div/div[3]").click()
driver.implicitly_wait(5)

# 스크롤 n회 내리기
def scroll_down(n):
    scroll_actions = driver.find_element(By.CSS_SELECTOR, 'body')
    for i in range(n):
        scroll_actions.send_keys(Keys.END)
        driver.implicitly_wait(5)
scroll_down(30)

# 썸네일 링크 n개 리스트 저장
thumb_list = []
for i in tqdm(range(1, 11)):
    try: 
        thumnail_format = "/html/body/div[3]/div[2]/div/div[1]/section/div[1]/div[2]/ul/li[{}]/div[1]/div/div[2]/a[1]/img".format(i)
        thumb_list.append(driver.find_element(By.XPATH, thumnail_format).get_attribute("src"))
    except: pass # 이미지가 없는 뉴스 패스
print(f"{len(thumb_list)}개 이미지 링크 확인")

# 이미지 저장 폴더 생성
folder_path = os.path.join(os.getcwd(), '웹크롤링_뉴스이미지')
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    
# 이미지 저장
cnt = 0
for link in tqdm(thumb_list):
    urlretrieve(link, folder_path + f"\{cnt}.jpg")
    cnt += 1
    time.sleep(0.3)
    
# 이미지 저장한 폴더 안의 파일 개수 확인
print(f"{len(os.listdir(folder_path))}개 이미지 저장")
    
# 크롬 창 종료
driver.quit()

100%|██████████| 10/10 [00:00<00:00, 50.97it/s]


10개 이미지 링크 확인


100%|██████████| 10/10 [00:03<00:00,  2.89it/s]


10개 이미지 저장


### test_03 | 와챠피디아

In [155]:
# 사이트 오픈
target_url = "https://pedia.watcha.com/ko-KR/?domain=movie" 
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.implicitly_wait(5) # 대기
driver.maximize_window() # 화면 최대화
driver.get(target_url)

In [156]:
# 팝업 종료
driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div/div/div[2]/span[1]").click()

# 박스오피스 순위 30개 불러오기
for i in range(6):
    driver.find_element(By.XPATH, "/html/body/div/div/div[1]/section/div/section/div[3]/section/div[2]/button").click()
    time.sleep(0.5)
    
# df 생성
movie_df = pd.DataFrame(columns=['title', 'link', 'rate', 'poster', 'reviews'])
# 이미지 저장 폴더 생성
folder_path = os.path.join(os.getcwd(), '웹크롤링_영화이미지')
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    
# 30개 영화에 대한 정보 모으기
for movie_cnt in tqdm(range(1, 31)):
    # 상세 페이지 이동
    driver.find_element(By.XPATH, f"/html/body/div[1]/div/div[1]/section/div/section/div[3]/section/div[1]/ul/li[{movie_cnt}]").click()
    driver.implicitly_wait(5) # 대기

    # 제목, 링크, 리뷰
    title = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/h1").text
    web_link = driver.current_url

    # 평점 있으면 저장, 없을 경우 "미개봉"
    try: rate = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[1]").text
    except: rate = "미개봉"

    # 포스터 저장
    poster_link = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[1]/div/div/img").get_attribute('src')
    poeter_image_path = folder_path + f"\{movie_cnt}.jpg"
    urlretrieve(poster_link, poeter_image_path)

    # 리뷰는 개수에 따라 저장
    reviews_list = []
    reviews_count = len(driver.find_elements(By.XPATH, "/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[2]/section[2]/ul/li"))
    for i in range(1, reviews_count+1):
        review = driver.find_elements(By.XPATH, f"/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[2]/section[2]/ul/li[{i}]/div/div[2]/a/div/div")[0].text
    if len(review) > 0:
        reviews_list.append(review)
    else: 
        driver.find_element(By.XPATH, f"/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[2]/section[2]/ul/li[{i}]/div/div[2]/span/button").click()
        time.sleep(0.3)
        review = driver.find_elements(By.XPATH, f"/html/body/div[1]/div/div[1]/section/div/div[2]/div/div[2]/section[2]/ul/li[{i}]/div/div[2]/a/div/div")[0].text
        reviews_list.append(review)
        
    # df에 차례대로 저장
    movie_df.loc[movie_cnt] = [title, web_link, rate, poeter_image_path, reviews_list]
    time.sleep(0.3)
    
    # 뒤로가기
    driver.back()

# 크롬 창 종료
driver.quit()

print(movie_df.shape, movie_df.head(), movie_df.tail(), sep = "\n")

100%|██████████| 30/30 [01:27<00:00,  2.92s/it]


(30, 5)
                 title                                             link rate  \
1                   파묘  https://pedia.watcha.com/ko-KR/contents/m53mNG2  3.5   
2  극장판 스파이 패밀리 코드: 화이트  https://pedia.watcha.com/ko-KR/contents/m5Zllo6  미개봉   
3               듄: 파트2  https://pedia.watcha.com/ko-KR/contents/mWvqG3e  4.2   
4                 댓글부대  https://pedia.watcha.com/ko-KR/contents/m5x1xva  미개봉   
5                탐정 말로  https://pedia.watcha.com/ko-KR/contents/mdMRRwy  미개봉   

                                    poster  \
1  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\1.jpg   
2  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\2.jpg   
3  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\3.jpg   
4  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\4.jpg   
5  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\5.jpg   

                                      reviews  
1              [장재현만큼 공포 장르에 뚝심있는 한국 감독이 없다.]  
2              [스파이 패밀리의 가족여행과 눈이 즐거운 쾌감 액션!]  
3  [구원이 아닌 지배를 갈망하는 메시아의 잠재적 재앙. 선택은 곧 강압이다.]  
4                            [김성철 손석규? 미쳣다

In [150]:
print(movie_df.shape, movie_df.head(), movie_df.tail(), sep = "\n")

(30, 5)
                 title                                             link rate  \
1                   파묘  https://pedia.watcha.com/ko-KR/contents/m53mNG2  3.5   
2  극장판 스파이 패밀리 코드: 화이트  https://pedia.watcha.com/ko-KR/contents/m5Zllo6  미개봉   
3               듄: 파트2  https://pedia.watcha.com/ko-KR/contents/mWvqG3e  4.2   
4                 댓글부대  https://pedia.watcha.com/ko-KR/contents/m5x1xva  미개봉   
5                탐정 말로  https://pedia.watcha.com/ko-KR/contents/mdMRRwy  미개봉   

                                    poster  \
1  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\1.jpg   
2  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\2.jpg   
3  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\3.jpg   
4  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\4.jpg   
5  c:\dqa\테스트자동화_Selenium\웹크롤링_영화이미지\5.jpg   

                                      reviews  
1              [장재현만큼 공포 장르에 뚝심있는 한국 감독이 없다.]  
2              [스파이 패밀리의 가족여행과 눈이 즐거운 쾌감 액션!]  
3  [구원이 아닌 지배를 갈망하는 메시아의 잠재적 재앙. 선택은 곧 강압이다.]  
4                            [김성철 손석규? 미쳣다